In [1]:
# !git clone https://github.com/LeelaChessZero/lczero-training.git

In [2]:
# %cd lczero-training/
# !pip install -r tf/requirements.txt

In [3]:
# %ls -l

In [4]:
# %cd libs
# !git clone https://github.com/LeelaChessZero/lczero-common.git

In [5]:
# %cd ..
# %ls -l
# !./init.sh

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
# %cd tf

In [8]:
#!/usr/bin/env python3
import argparse
import os
import yaml
import tfprocess
from net import Net


In [9]:

cfg_path = "../128x10-t60-2.yaml"
net_path = "../128x10-t60-2-5300.pb.gz"
ignore_errors = False

with open(cfg_path,'r') as f:
  cfg = yaml.safe_load(f.read())

In [10]:


print(yaml.dump(cfg, default_flow_style=False))
# START_FROM = args.start

tfp = tfprocess.TFProcess(cfg)
tfp.init_net_v2()
tfp.replace_weights_v2(net_path, ignore_errors)
# tfp.global_step.assign(START_FROM)

# root_dir = os.path.join(cfg['training']['path'], cfg['name'])
# if not os.path.exists(root_dir):
#     os.makedirs(root_dir)
# # tfp.manager.save(checkpoint_number=START_FROM)
# print("Wrote model to {}".format(tfp.manager.latest_checkpoint))


dataset:
  allow_less_chunks: true
  input_test: /temp/sergio-v/t60/test/rescored/*/
  input_train: /temp/sergio-v/t60/train/rescored/*/
  num_chunks: 100000000
  test_workers: 8
  train_ratio: 0.9
  train_workers: 16
gpu: all
model:
  filters: 128
  moves_left: v1
  residual_blocks: 10
  se_ratio: 4
  value_channels: 32
name: 128x10-2
training:
  batch_size: 4096
  checkpoint_steps: 10000
  lr_boundaries:
  - 100
  lr_values:
  - 0.0002
  - 0.0002
  mask_legal_moves: true
  max_grad_norm: 3.0
  memory_limit: 9000
  moves_left_gradient_flow: 1.0
  moves_left_loss_weight: 0.1
  num_batch_splits: 4
  num_test_positions: 100000
  path: /home/s/sergio-v/project/new/networks
  policy_loss_weight: 1.0
  q_ratio: 0.0
  reg_loss_weight: 1.0
  renorm: true
  renorm_max_d: 0.0
  renorm_max_r: 1.0
  shuffle_size: 500000
  swa: true
  swa_max_n: 10
  swa_output: true
  swa_steps: 25
  test_steps: 1000
  total_steps: 1000000000
  train_avg_report_steps: 100
  trainstop_path: /temp/sergio-v/trainsto

In [11]:
cfg['dataset']['input_train'] = "../../drive/MyDrive/icg-chess/supervised-0/*"
cfg['dataset']['input_test'] = "../../drive/MyDrive/icg-chess/supervised-0/*"

In [12]:
import glob
for d in glob.glob(cfg['dataset']['input_train']):
  print(d)

In [13]:
from train import get_latest_chunks, get_all_chunks, get_chunks

print(yaml.dump(cfg, default_flow_style=False))

num_chunks = cfg['dataset']['num_chunks']
allow_less = cfg['dataset'].get('allow_less_chunks', False)
train_ratio = cfg['dataset']['train_ratio']
experimental_parser = cfg['dataset'].get('experimental_v5_only_dataset',
                                          False)
num_train = int(num_chunks * train_ratio)
num_test = num_chunks - num_train
sort_type = cfg['dataset'].get('sort_type', 'mtime')
if sort_type == 'mtime':
    sort_key_fn = os.path.getmtime
elif sort_type == 'number':
    sort_key_fn = game_number_for_name
elif sort_type == 'name':
    sort_key_fn = identity_function
else:
    raise ValueError('Unknown dataset sort_type: {}'.format(sort_type))
if 'input_test' in cfg['dataset']:
    train_chunks = get_latest_chunks(cfg['dataset']['input_train'],
                                      1, allow_less, sort_key_fn)
    test_chunks = get_latest_chunks(cfg['dataset']['input_test'], 1,
                                    allow_less, sort_key_fn)
else:
    chunks = get_latest_chunks(cfg['dataset']['input'], num_chunks,
                                allow_less, sort_key_fn)
    if allow_less:
        num_train = int(len(chunks) * train_ratio)
        num_test = len(chunks) - num_train
    train_chunks = chunks[:num_train]
    test_chunks = chunks[num_train:]

shuffle_size = cfg['training']['shuffle_size']
total_batch_size = cfg['training']['batch_size']
batch_splits = cfg['training'].get('num_batch_splits', 1)
train_workers = cfg['dataset'].get('train_workers', None)
test_workers = cfg['dataset'].get('test_workers', None)
if total_batch_size % batch_splits != 0:
    raise ValueError('num_batch_splits must divide batch_size evenly')
split_batch_size = total_batch_size // batch_splits
# Load data with split batch size, which will be combined to the total batch size in tfprocess.
ChunkParser.BATCH_SIZE = split_batch_size

dataset:
  allow_less_chunks: true
  input_test: ../../drive/MyDrive/icg-chess/supervised-0/*
  input_train: ../../drive/MyDrive/icg-chess/supervised-0/*
  num_chunks: 100000000
  test_workers: 8
  train_ratio: 0.9
  train_workers: 16
gpu: all
model:
  filters: 128
  moves_left: v1
  residual_blocks: 10
  se_ratio: 4
  value_channels: 32
name: 128x10-2
training:
  batch_size: 4096
  checkpoint_steps: 10000
  lr_boundaries:
  - 100
  lr_values:
  - 0.0002
  - 0.0002
  mask_legal_moves: true
  max_grad_norm: 3.0
  memory_limit: 9000
  moves_left_gradient_flow: 1.0
  moves_left_loss_weight: 0.1
  num_batch_splits: 4
  num_test_positions: 100000
  path: /home/s/sergio-v/project/new/networks
  policy_loss_weight: 1.0
  q_ratio: 0.0
  reg_loss_weight: 1.0
  renorm: true
  renorm_max_d: 0.0
  renorm_max_r: 1.0
  shuffle_size: 500000
  swa: true
  swa_max_n: 10
  swa_output: true
  swa_steps: 25
  test_steps: 1000
  total_steps: 1000000000
  train_avg_report_steps: 100
  trainstop_path: /temp/

IndexError: list index out of range